In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from catch22_master.wrap_Python.catch22.catch22 import *
from keras.preprocessing.sequence import TimeseriesGenerator
import gc

import glob
from tqdm import tqdm, tqdm_notebook

Using TensorFlow backend.


In [2]:
def readFiles():
    fileDic = {}
    files = [file for file in glob.glob('Data/*.pkl')]
    for name in tqdm(files):
        file = pickle.load(open(name, 'rb'), encoding = 'latin1')
        fileDic[name.split('.')[0].split('/')[-1]] = file
    return fileDic

In [3]:
fileDic = readFiles()

100%|██████████| 15/15 [01:26<00:00,  5.47s/it]


In [4]:
ts_seg = {'chest':{'ECG':{'wsiz':42000, 'strid':3000, 'num_s':1},
                   'EDA':{'wsiz':42000, 'strid':3000, 'num_s':1},
                   'Temp':{'wsiz':42000, 'strid':3000, 'num_s':1},
                   'ACC':{'wsiz':42000, 'strid':3000, 'num_s':3},
                   'EMG':{'wsiz':42000, 'strid':3000, 'num_s':1}, 
                  'Resp':{'wsiz':42000, 'strid':3000, 'num_s':1}},
         'wrist':{'BVP':{'wsiz':3840, 'strid':16, 'num_s':1},
                   'EDA':{'wsiz':240, 'strid':1, 'num_s':1},
                   'TEMP':{'wsiz':240, 'strid':1, 'num_s':1},
                   'ACC':{'wsiz':160, 'strid':8, 'num_s':3}}, 
         'Label':{'wsiz':42000, 'strid':175, 'num_s':1}}

In [5]:
''' This module has essential functions supporting
fast and effective computation of permutation entropy and
its different variations.'''
import itertools
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean


def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector represnting the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector represnting the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of ordinal patterns'''
    time_series = ts
    possible_permutations = list(itertools.permutations(range(embdim)))
    lst = list()
    for i in range(len(time_series) - embdelay * (embdim - 1)):
        sorted_index_array = list(np.argsort(time_series[i:(embdim+i)]))
        lst.append(sorted_index_array)
    lst = np.array(lst)
    element, freq = np.unique(lst, return_counts = True, axis = 0)
    freq = list(freq)
    if len(freq) != len(possible_permutations):
        for i in range(len(possible_permutations)-len(freq)):
            freq.append(0)
        return(freq)
    else:
        return(freq)

def p_entropy(op):
    ''' This function computes the permutation entropy of a given ordinal pattern.
    USAGE: permutation_entropy(op)
    ARGS: op = Numeric vector represnting the ordinal pattern
    OUTPUT: A numeric value representing permutation entropy'''
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)

def complexity(op):
    ''' This function computes the complexity of a time series defined as: Comp_JS = Q_o * JSdivergence * pe
    Q_o = Normalizing constant
    JSdivergence = Jensen-Shannon divergence
    pe = permutation entopry
    ARGS: ordinal pattern'''
    pe = p_entropy(op)
    constant1 = (0.5+((1 - 0.5)/len(op)))* np.log(0.5+((1 - 0.5)/len(op)))
    constant2 = ((1 - 0.5)/len(op))*np.log((1 - 0.5)/len(op))*(len(op) - 1)
    constant3 = 0.5*np.log(len(op))
    Q_o = -1/(constant1+constant2+constant3)

    temp_op_prob = np.divide(op, sum(op))
    temp_op_prob2 = (0.5*temp_op_prob)+(0.5*(1/len(op)))
    JSdivergence = (s_entropy(temp_op_prob2) - 0.5 * s_entropy(temp_op_prob) - 0.5 * np.log(len(op)))
    Comp_JS = Q_o * JSdivergence * pe
    return(Comp_JS)

def weighted_ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the weighted ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: weighted_ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector represnting the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of weighted ordinal patterns'''
    time_series = ts
    possible_permutations = list(itertools.permutations(range(embdim)))
    temp_list = list()
    wop = list()
    for i in range(len(time_series) - embdelay * (embdim - 1)):
        Xi = time_series[i:(embdim+i)]
        Xn = time_series[(i+embdim-1): (i+embdim+embdim-1)]
        Xi_mean = np.mean(Xi)
        Xi_var = (Xi-Xi_mean)**2
        weight = np.mean(Xi_var)
        sorted_index_array = list(np.argsort(Xi))
        temp_list.append([''.join(map(str, sorted_index_array)), weight])
    result = pd.DataFrame(temp_list,columns=['pattern','weights'])
    freqlst = dict(result['pattern'].value_counts())
    for pat in (result['pattern'].unique()):
        wop.append(np.sum(result.loc[result['pattern']==pat,'weights'].values))
    return(wop)



In the next section, one of the patients records and the irrelevant labels are dropped  

In [6]:
for user in tqdm(fileDic):
    if user == 'S7':
        continue
    user_data = fileDic[user].copy()
    mask = np.isin(user_data['label'], [0,4,5,6,7], invert=True)
    user_data['label'] = user_data['label'][mask]
    keys = ['Resp', 'ECG', 'ACC', 'Temp', 'EDA', 'EMG']
    for j in tqdm_notebook(range(len(keys))):
        k = keys[j]
        user_data['signal']['chest'][k] = user_data['signal']['chest'][k][mask]
        data_transformed = {'x':{}, 'x1':{},'x2':{}}#, 'x3':{},'x4':{}, 'x5':{}}
        data_gen = TimeseriesGenerator(user_data['signal']['chest'][k], user_data['label'],
                                            length = ts_seg['chest'][k]['wsiz'], stride = ts_seg['chest'][k]['strid'] ,
                                            batch_size=1)

        internal_df = np.vstack([np.transpose(x, axes=(0,2,1)) for x, y in data_gen])
        internal_label = np.vstack([np.transpose([y], axes=(1,0)) for x, y in data_gen])

        # for each sensor from each error we calculate the 3 features disscussed above
        for num_sensor in tqdm_notebook(range(ts_seg['chest'][k]['num_s'])):
            data_transformed['x'][str(num_sensor)] = [p_entropy(ordinal_patterns(internal_df[i,num_sensor,:],5,1)) for i in tqdm_notebook(range(internal_df.shape[0]))]
            data_transformed['x1'][str(num_sensor)] = [complexity(ordinal_patterns(internal_df[i,num_sensor,:],5,1)) for i in tqdm_notebook(range(internal_df.shape[0]))]
            data_transformed['x2'][str(num_sensor)] = [s_entropy(ordinal_patterns(internal_df[i,num_sensor,:],5,1)) for i in tqdm_notebook(range(internal_df.shape[0]))]
#             data_transformed['x3'][str(num_sensor)] = [p_entropy(weighted_ordinal_patterns(internal_df[i,num_sensor,:],5,1)) for i in tqdm_notebook(range(internal_df.shape[0]))]
#             data_transformed['x4'][str(num_sensor)] = [complexity(weighted_ordinal_patterns(internal_df[i,num_sensor,:],5,1)) for i in tqdm_notebook(range(internal_df.shape[0]))]
#             data_transformed['x5'][str(num_sensor)] = [s_entropy(weighted_ordinal_patterns(internal_df[i,num_sensor,:],5,1)) for i in tqdm_notebook(range(internal_df.shape[0]))]

        data_transformed['y'] = internal_label

        with open(user + '_' + 'chest' + "_" + k +'_updated_transformed.pickle', 'wb') as f:
            pickle.dump(data_transformed, f, protocol=pickle.HIGHEST_PROTOCOL)

        del data_transformed, data_gen, internal_df, internal_label
        gc.collect()



  0%|          | 0/15 [00:00<?, ?it/s]

 13%|█▎        | 2/15 [47:57<5:11:42, 1438.63s/it]

 20%|██        | 3/15 [1:35:45<6:13:29, 1867.46s/it]

 27%|██▋       | 4/15 [2:25:02<6:42:18, 2194.41s/it]

 33%|███▎      | 5/15 [3:12:40<6:38:55, 2393.54s/it]

...

 73%|███████▎  | 11/15 [8:06:30<3:12:16, 2884.08s/it]

 80%|████████  | 12/15 [8:56:37<2:26:03, 2921.17s/it]

 87%|████████▋ | 13/15 [9:48:25<1:39:14, 2977.05s/it]

 93%|█████████▎| 14/15 [10:40:19<50:18, 3018.24s/it] 

100%|██████████| 15/15 [11:31:23<00:00, 3032.01s/it]
